In [2]:
import os
result=[]
def search(dirname):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                ext = os.path.splitext(full_filename)[-1]
                if ext == '.jpg': 
                    #print(full_filename)
                    result.append(full_filename)
    except PermissionError:
        pass

search("/home/lab/dataset/skin_dataset/sfa_nonskin")

In [3]:
import cv2
test_result=[]
for i in range(100):
    img = cv2.imread(result[i])
    for j in range(len(img)-3):
        for k in range(len(img[0])-3):
            make_data=img[j:j+3,k:k+3]
            test_result.append(make_data)

In [4]:
import numpy as np
test_result=np.array(test_result)

In [6]:
import os
result1=[]
def search(dirname):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                ext = os.path.splitext(full_filename)[-1]
                if ext == '.jpg': 
                    #print(full_filename)
                    result1.append(full_filename)
    except PermissionError:
        pass

search("/home/lab/dataset/skin_dataset/sfa_skin")

In [7]:
test_result1=[]
for i in range(100):
    img = cv2.imread(result1[i])
    for j in range(len(img)-3):
        for k in range(len(img[0])-3):
            make_data=img[j:j+3,k:k+3]
            test_result1.append(make_data)

In [8]:
test_result1=np.array(test_result1)

In [9]:
def YCbCrTransform(x):
    #================= exponancial function ==================#
    exponancial=[[65.738, 129.057, 25.064],
                 [-37.945, -74.494, 112.439],
                 [112.439, -94.154, -18.285]]
    exponancial=np.matrix(exponancial)
    exponancial=(1/256)*exponancial
    
    A=[[16],[128],[128]]
    A=np.matrix(A)
    #=========================================================#
    x=np.matrix(x)
    x=np.transpose(x)
    Answer=A+exponancial*x
    Answer=np.array(Answer)
    Answer=Answer.tolist()
    Cb=Answer[1][0]
    Cr=Answer[2][0]
    
    return Cb,Cr
def Make_redemention(x):
    before_result=[]
    for i in range(x.shape[0]):    #x.shape[0]
        before_result1=[]
        for j in range(x.shape[1]):
            before_result2=[]
            for k in range(x.shape[2]):
                before_result3=[]
                before=x[i][j][k]
                Cb,Cr=YCbCrTransform(before)
                before_result3.append(before[0])
                before_result3.append(before[1])
                before_result3.append(before[2])
                before_result3.append(Cb)
                before_result3.append(Cr)
                before_result2.append(before_result3)
            before_result1.append(before_result2)
        before_result.append(before_result1)
    return before_result
def reshape_demension(x):
    make_result=[]
    for i in range(x.shape[0]):
        make_result1=[]
        for j in range(x.shape[1]):
            for k in range(x.shape[2]):
                make_result1.append(x[i][j][k])
        make_result.append(make_result1)
    return make_result

In [10]:
AAA=Make_redemention(test_result)
BBB=Make_redemention(test_result1)

In [11]:
AAA=np.array(AAA,dtype=np.uint8)
BBB=np.array(BBB,dtype=np.uint8)

In [12]:
AAA1=reshape_demension(AAA)
BBB1=reshape_demension(BBB)

In [13]:
AAA1=np.array(AAA1)
BBB1=np.array(BBB1)

In [14]:
AAA1= (AAA1*(1/256)-0.5)/0.5 #non-skin
BBB1= (BBB1*(1/256)-0.5)/0.5 #Skin

In [15]:
BBB1[15355]

array([[-0.03125  ,  0.4453125,  0.9609375,  0.296875 , -0.25     ],
       [-0.0390625,  0.4375   ,  0.96875  ,  0.296875 , -0.25     ],
       [-0.015625 ,  0.4609375,  0.9921875,  0.296875 , -0.25     ],
       [-0.0078125,  0.46875  ,  0.984375 ,  0.296875 , -0.25     ],
       [-0.03125  ,  0.4453125,  0.9609375,  0.296875 , -0.25     ],
       [-0.0234375,  0.453125 ,  0.96875  ,  0.296875 , -0.25     ],
       [ 0.0078125,  0.484375 ,  0.9921875,  0.2890625, -0.25     ],
       [ 0.       ,  0.4765625,  0.9765625,  0.2890625, -0.25     ],
       [ 0.015625 ,  0.4921875,  0.9921875,  0.2890625, -0.25     ]])

In [16]:
AAA1.shape, BBB1.shape
label=[]
for i in range(39200):
    if i >19599:
        label.append(0)
    else :
        label.append(1)
input_data=[]
for i in range(2):
    if i == 0:
        for j in range(len(AAA1)):
            input_data.append(AAA1[j])
    else :
        for j in range(len(BBB1)):
            input_data.append(BBB1[j])
input_data=np.array(input_data)
label=np.array(label)

In [17]:
input_data.shape, label.shape

((39200, 9, 5), (39200,))

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
import seaborn as sns 
import matplotlib.pyplot as plt

In [20]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [21]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [22]:
trainset,testset=torch.utils.data.random_split(dataset,[31360,7840])
partition={'train':trainset,'test':testset}

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
import seaborn as sns 
import matplotlib.pyplot as plt

In [24]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
class test(nn.Module):
    def __init__(self):
        super(test,self).__init__()
        A = [[1, 1, 0, 1, 1, 0, 0, 0, 0],
             [1, 1, 1, 1, 1, 1, 0, 0, 0],
             [0 ,1, 1, 0, 1, 1, 0, 0, 0],
             [1, 1, 0, 1, 1, 0, 1, 1, 0],
             [1, 1, 1, 1, 1, 1, 1, 1, 1],
             [0, 1, 1, 0, 1, 1, 0, 1, 1],
             [0, 0, 0, 1, 1, 0, 1, 1, 0],
             [0, 0, 0, 1, 1, 1, 1, 1, 1],
             [0, 0, 0, 0, 1, 1, 0, 1, 1]]
        adj=torch.FloatTensor(A)
        adj=adj.cuda()
        self.adj=adj
        self.W= nn.Parameter(torch.detach(torch.rand(5,48)).requires_grad_(True))
        self.act = nn.Sigmoid()
        self.Linear=nn.Linear(432,10)
        self.Linear1=nn.Linear(10,2)
#ReLU 쓰니까 out 이 2번째에서 다 0값이 나옴.
        
    def forward(self,x):
        out = torch.matmul(x,self.W)
        out = torch.matmul(self.adj,out)
        out = out.view(-1,432)
        out = self.Linear(out)
        out = self.act(out)
        out = self.Linear1(out)
        #out=F.softmax(out,dim=1)
        
        return out

In [25]:
def train(net,partition,optimizer,creiterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=256,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [27]:
def test1(net, partition, args):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=256, 
                                             shuffle=True, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [ ]:
net = test()
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_accs=[]
for epoch in range(200):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    test_acc = test1(net,partition,criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},test_acc : {:2.2f}'.format(te-ts,train_loss,train_acc,test_acc))

time : 0.50, train_loss : 0.41, train_acc : 85.47,test_acc : 88.06
time : 0.51, train_loss : 0.29, train_acc : 89.08,test_acc : 89.30
time : 0.49, train_loss : 0.25, train_acc : 90.15,test_acc : 92.54
time : 0.52, train_loss : 0.23, train_acc : 92.11,test_acc : 92.40
time : 0.48, train_loss : 0.21, train_acc : 92.37,test_acc : 91.52
time : 0.54, train_loss : 0.19, train_acc : 92.63,test_acc : 91.49
time : 0.49, train_loss : 0.18, train_acc : 92.61,test_acc : 91.86
time : 0.50, train_loss : 0.18, train_acc : 92.49,test_acc : 91.93
time : 0.50, train_loss : 0.17, train_acc : 92.77,test_acc : 92.63
time : 0.49, train_loss : 0.17, train_acc : 92.89,test_acc : 92.26
time : 0.48, train_loss : 0.16, train_acc : 92.66,test_acc : 92.44
time : 0.54, train_loss : 0.16, train_acc : 92.91,test_acc : 92.21
time : 0.51, train_loss : 0.16, train_acc : 93.06,test_acc : 92.56
time : 0.50, train_loss : 0.15, train_acc : 93.01,test_acc : 93.20
time : 0.48, train_loss : 0.15, train_acc : 93.23,test_acc : 9